In [1]:
import re
!pip install biopython
from Bio import SeqIO

In [2]:
def show_lengths(fname):
    def extract_len(line):
        pattern = r'^>[a-z]+[0-9]+_len([0-9]+)_cov[0-9]+'
        return int(re.search(pattern, line).group(1))

    with open(fname) as file:
        lens = sorted([extract_len(line) for line in file if line.startswith(">")])
    
    sumlen = sum(lens)
    part_sum = 0
    n50 = None
    for l in reversed(lens):
        part_sum += l
        if part_sum * 2 >= sumlen:
            n50 = l
            break

    print(
        f"{fname}",
        f"count:     {len(lens)}",
        f"total len: {sumlen}",
        f"longest:   {lens[-1]}",
        f"n50:       {n50}",
        sep="\n"
    )

### Анализ контигов

In [3]:
show_lengths("../data/out_contig.fa")

../data/out_contig.fa
count:     617
total len: 3925575
longest:   179307
n50:       47611


In [4]:
show_lengths("../data/out_scaffold.fa")

../data/out_scaffold.fa
count:     98
total len: 3869816
longest:   383582
n50:       173397


### Количество гэпов

In [5]:
def show_longest(fname, label, out_fname):
    seqs = list(SeqIO.parse(fname, "fasta"))
    
    longest = max(seqs, key=len)
    gaps = re.findall(r"N+", str(longest.seq))
    print(
        f"{label}",
        f"gap count:        {len(gaps)}",
        f"gap total length: {sum(map(len, gaps))}",
        sep="\n"
    )
    
    SeqIO.write([longest], out_fname, "fasta")
    
show_longest("../data/out_scaffold.fa", "not closed", "../data/longest_not_closed.fa")
print()
show_longest("../data/out_scaffold_closed.fa", "gaps closed", "../data/longest_closed.fa")

not closed
gap count:        2
gap total length: 25

gaps closed
gap count:        0
gap total length: 0
